In [ ]:
import os
import json


def list_folders_in_json_folder(json_folder):
    """
    List all folders inside the specified JSON folder.
    """
    try:
        # Get a list of all entries in the json_folder
        all_entries = os.listdir(json_folder)
        
        # Filter only the directories (folders)
        folders = [entry for entry in all_entries if os.path.isdir(os.path.join(json_folder, entry))]
        
        # Return the list of folder names
        return folders
    except Exception as e:
        print(f"Error accessing {json_folder}: {e}")
        return []
    
def extract_json_metadata(image_filename, json_folder, search_key=None):
    """
    Extract 'photoTakenTime' metadata from a JSON file in a separate folder, using the same base name as the image.
    """
    json_files = []
    json_path = f"{os.path.splitext(image_filename)[0]}.json"  # Find corresponding JSON
    #print(json_path)
    # Walk through the JSON folder to find the corresponding JSON file
    for filenames in os.listdir(json_folder):
            if filenames.startswith("takeout-") and not filenames.lower().endswith("zip"):  # Match base name of the image file
                file_path = os.path.join(filenames, json_path)
                # Check for the specific key inside the JSON file (optional)
                if os.path.exists(file_path):
                    if search_key:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            try:
                                file_content = json.load(f)
                                if search_key in file_content:
                                    print(f"Found key '{search_key}' in {file_path}")
                                    return file_content.get(search_key, None)  # Return the photoTakenTime if found
                            except json.JSONDecodeError:
                                print(f"Error decoding JSON in file: {file_path}")
                    
                    json_files.append(file_path)

    return json_files 



    
def process_images_in_folder(image_folder, json_folder):
    """
    Process all images in a folder to extract their photoTakenTime metadata from a separate JSON folder.
    """
    results = []
    for file_name in os.listdir(image_folder):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):  # Check for image files
            image_path = os.path.join(image_folder, file_name)
            #print(image_path)
            photo_taken_time = extract_json_metadata(file_name, json_folder,"photoTakenTime")  # Extract photoTakenTime from JSON
            results.append({"image_path": image_path, "photo_taken_time": photo_taken_time})
    return results

def process_photos_in_year_folder(base_folder, json_folder):
    """
    Process folders for each year (starting with Photos_20) to extract photoTakenTime from images using JSON from a separate folder.
    """
    year_data = {}
    for year_folder in os.listdir(base_folder):
        year_path = os.path.join(base_folder, year_folder)
        if os.path.isdir(year_path) and year_folder.startswith("Photos_20"):  # Look for Photos_20** folders
            year_data[year_folder] = {}
            for month_folder in os.listdir(year_path):
                month_path = os.path.join(year_path, month_folder)
                if os.path.isdir(month_path):  # Process each month folder
                    #print(f"Processing Year: {year_folder}, Month: {month_folder}")
                    month_data = process_images_in_folder(month_path, json_folder)  # Extract photoTakenTime from each image
                    year_data[year_folder][month_folder] = month_data
    return year_data

# Base folder containing year/month photo folders
base_folder = r"C:\Users\Lenovo\Desktop"  # Adjust path to your actual base folder

# Separate folder where JSON files are stored
json_folder = r"C:\Users\Lenovo\Desktop\google_takeout"  # Adjust path to your actual JSON folder

# The key you're searching for inside the JSON files (optional)
search_key = "photoTakenTime"

# Start searching for JSON files on Desktop and check for the specified key
#found_json_files = search_json_in_directory(base_folder, search_key)

# Process and store results for photoTakenTime extraction
photo_data = process_photos_in_year_folder(base_folder, json_folder)

# List folders inside the JSON folder
folders_in_json_folder = list_folders_in_json_folder(json_folder)

# Print the list of folders
print("Folders in JSON folder:", folders_in_json_folder)

# Print out all found JSON files
print(f"Found JSON files: {found_json_files}")

# Save the results to a JSON file
output_file = "photo_taken_times.json"
with open(output_file, "w") as f:
    json.dump(photo_data, f, indent=4)

print(f"Photo taken times saved to {output_file}.")
